In [13]:
using LinearAlgebra

# Approximation numérique du gradient
function grad(f, x)
    e = 1e-6
    g = zeros(length(x))
    for i in 1:length(x)
        h = zeros(length(x))
        h[i] = e
        g[i] = (f(x + h) - f(x - h)) / (2 * e)
    end
    return g
end

# Approximation de la matrice Hessienne
function hessian(f, x)
    e = 1e-6
    n = length(x)
    H = zeros(n, n)
    for i in 1:n
        for j in 1:n
            dx1, dx2, dx3 = zeros(n), zeros(n), zeros(n)
            dx1[i] += e; dx1[j] += e
            dx2[i] += e; dx3[j] += e
            H[i, j] = (f(x + dx1) - f(x + dx2) - f(x + dx3) + f(x)) / (e^2)
        end
    end
    return H
end

# Résolution des conditions KKT
function kkt_solver(f, constraints, x0; tol=1e-6, max_iter=100)
    n = length(x0)               # Dimensions de x
    m = length(constraints)      # Nombre de contraintes
    x = x0                       # Point initial
    λ = zeros(m)                 # Multiplicateurs de Lagrange initiaux

    for iter in 1:max_iter
        # Construction du gradient de la Lagrangienne
        grad_L = grad(f, x)
        for (i, c) in enumerate(constraints)
            grad_L += λ[i] * grad(c, x)
        end

        # Construction de la matrice Hessienne et Jacobienne
        H = hessian(f, x)
        J = hcat([grad(c, x) for c in constraints]...)  # Jacobienne des contraintes
        KKT_matrix = [H J; J' zeros(m, m)]
        KKT_rhs = [-grad_L; -[c(x) for c in constraints]]

        # Résolution du système KKT
        delta = nothing
        try
            delta = KKT_matrix \ KKT_rhs
        catch e
            if e isa SingularException
                println("Matrice KKT singulière : problème mal posé.")
                break
            else
                rethrow(e)
            end
        end

        if delta === nothing
            println("Erreur dans la résolution du système KKT.")
            break
        end

        dx = delta[1:n]
        dλ = delta[n+1:end]

        # Mise à jour des variables
        x += dx
        λ += dλ

        # Vérification du critère de convergence
        if norm(dx) < tol && norm([c(x) for c in constraints]) < tol
            println("Convergence atteinte après $iter itérations.")
            return x, λ
        end
    end

    println("Aucune convergence après $max_iter itérations.")
    return x, λ
end

# Exemple d'utilisation
function f(x)
    return x[1]^2 + x[2]^2
end

function h1(x)
    return x[1] + x[2] - 1
end

function h2(x)
    return (x[1]^2) / 2 + x[2]^2 - 1
end

x0 = [0.5, 0.5]
constraints = [h1, h2]

# Résolution
x_sol, λ_sol = kkt_solver(f, constraints, x0)
println("Solution : ", x_sol)
println("Multiplicateurs de Lagrange : ", λ_sol)


Convergence atteinte après 6 itérations.
Solution : [-8.392108812130486e-15, 1.0000000000000084]
Multiplicateurs de Lagrange : [-1.0619993881364918e-7, -0.9999998408071087]
